In [238]:
import random
import pandas as pd
import numpy as np

from collections import Counter

In [242]:
class StudentsMatching:
    def __init__(self, num_students, num_profiles, perc_of_target,
                 perc_of_contract, debts_limit, first_pref, first_pref_target):
        self.num_students = num_students
        self.num_profiles = num_profiles
        self.perc_of_target = perc_of_target
        self.perc_of_contract = perc_of_contract
        self.debts_limit = debts_limit
        self.first_pref = first_pref
        self.first_pref_target = first_pref_target
        
        self.data_s = pd.DataFrame(index=list(range(self.num_students)))
        self.data_p = pd.DataFrame(index=list(range(self.num_profiles)))
        
        self.profiles_matches = {prof: [] for prof in range(self.num_profiles)}
        self.student_matches = {student: None for student in range(self.num_students)}

        self.generate_student_labels()
        self.generate_debts()
        self.generate_rating()
        self.generate_profiles()
        self.generate_student_preferences()
        self.generate_profile_preferences()
        
        self.priorities, self.priorities_target, self.mandatory_profiles = self.calculate_mandatory_profiles()
        self.reserve_size = self.calculate_reserve_size()
        self.reserve_students = self.fill_reserve()
        self.unmatched_students = self.gale_shapley()

    def generate_student_labels(self):
        num_target_students = int(self.perc_of_target * self.num_students)
        self.data_s['target'] = 0
        rand_ind = np.random.choice(self.data_s.index, size=num_target_students, replace=False)
        self.data_s.loc[rand_ind, 'target'] = 1
        
        num_contract_students = int(self.perc_of_contract * self.num_students)
        self.data_s['contract'] = 0
        rand_ind = np.random.choice(self.data_s.index, size=num_contract_students, replace=False)
        self.data_s.loc[rand_ind, 'contract'] = 1

    def generate_debts(self):
        def generate_single_debt():
            return random.choices(range(8), weights=[10, 5, 3, 3, 2, 2, 1, 1], k=1)[0]

        self.data_s['debts'] = [generate_single_debt() for _ in self.data_s.index]

    def generate_rating(self):
        self.data_s['rating'] = [random.randint(0, 500) for _ in self.data_s.index]
    
    def generate_profiles(self):
        self.data_p['min_num_groups'] = 0
        self.data_p['max_num_groups'] = max_num_groups
        self.data_p['min_group_size'] = min_group_sizes
        self.data_p['max_group_size'] = max_group_sizes
        self.data_p['quota'] = self.data_p['max_group_size'] * self.data_p['max_num_groups']
        
    def generate_student_preferences(self):
        preferences_s = {}
        for student in self.data_s.index:
            if self.data_s['debts'][student] > self.debts_limit:
                preferences_s[student] = []
            else:
                preferences_s[student] = random.sample(list(self.data_p.index), k=self.num_profiles)
        self.data_s['preferences'] = self.data_s.index.map(preferences_s)
        
    def generate_profile_preferences(self):
        preferences_p = {}
        target_students_sorted = sorted(self.data_s[self.data_s['target'] == 1].index, key=lambda s: -self.data_s['rating'][s])
        other_students = sorted(
            self.data_s[(self.data_s['target'] == 0) & (self.data_s['debts'] <= self.debts_limit)].index,
            key=lambda s: (-self.data_s['rating'][s], self.data_s['contract'][s])
        )
        debt_students = sorted(self.data_s[self.data_s['debts'] > self.debts_limit].index, key=lambda s: -self.data_s['rating'][s])
        preferences_p = target_students_sorted + other_students + debt_students
        self.data_p['preferences'] = [preferences_p] * len(self.data_p)
    
    def calculate_mandatory_profiles(self):
        priorities = Counter([x[0] for x in self.data_s['preferences'] if x])
        priorities_target = Counter([x[0] for x in self.data_s[self.data_s['target'] == 1]['preferences'] if x])
        mandatory_profiles = list(set(
            [profile for profile, count in priorities.items() if count > self.first_pref] +
            [profile for profile, count in priorities_target.items() if count > self.first_pref_target]
        ))
        self.data_p.loc[self.data_p.index.isin(mandatory_profiles), 'min_num_groups'] = 1
        num_mandatory_profiles = len(mandatory_profiles)
        return priorities, priorities_target, mandatory_profiles
        
    def calculate_reserve_size(self):
        reserve_size = (self.data_p['min_group_size'] * self.data_p['min_num_groups']).sum()
        return reserve_size

    def fill_reserve(self):
        reserve_size = self.calculate_reserve_size()
        remaining_students = self.data_s[self.data_s['target'] == 0].index.tolist()
        reserve_students = self.data_s[self.data_s['preferences'].apply(lambda x: x == [])].sort_values(by='rating').index.tolist()
        if len(reserve_students) < reserve_size:
            needed_students_count = reserve_size - len(self.reserve_students)
            remaining_sorted_students = self.data_s[self.data_s['target'] == 0].sort_values(by='rating')
            students_to_add = remaining_sorted_students.head(needed_students_count)
            reserve_students += students_to_add.index.tolist()
        return reserve_students
    
    def gale_shapley(self):
        free_students = [s for s in self.data_s.index if s not in self.reserve_students]  
        proposed = {student: 0 for student in range(self.num_students)}
        while free_students:
            student = free_students[0]
            student_pref = self.data_s.loc[student, 'preferences']
            prof_index = proposed[student]
            if prof_index < len(student_pref):  
                prof = student_pref[prof_index]
                proposed[student] += 1
                if len(self.profiles_matches[prof]) < self.data_p['quota'][prof]:
                    self.profiles_matches[prof].append(student)
                    self.student_matches[student] = prof
                    free_students.pop(0)
                    if prof in self.mandatory_profiles and self.reserve_students:
                        student_from_reserve = self.reserve_students.pop(0)
                        free_students.append(student_from_reserve)
                else:
                    current_student = self.profiles_matches[prof][-1]
                    prof_pref = self.data_p['preferences'][prof]
                    if prof_pref.index(student) < prof_pref.index(current_student):
                        self.profiles_matches[prof].pop()
                        self.profiles_matches[prof].append(student)
                        self.student_matches[student] = prof
                        self.student_matches[current_student] = None
                        free_students.pop(0)
                        free_students.append(current_student)
            else:
                free_students.pop(0)
        unmatched_students = [s for s in range(self.num_students) if self.student_matches[s] is None]
        return unmatched_students

In [243]:
num_students = 450
num_profiles = 8

max_num_groups = [2, 2, 4, 2, 9, 3, 3, 4]
max_group_sizes = [28, 30, 30, 34, 32, 30, 29, 32]
min_group_sizes = [15, 20, 18, 20, 22, 19, 20, 21]

perc_of_target = 0.05
perc_of_contract = 0.3
debts_limit = 5

first_pref = 55
first_pref_target = 5

In [244]:
matching = StudentsMatching(num_students, num_profiles, perc_of_target,
                            perc_of_contract, debts_limit, first_pref, first_pref_target)

In [245]:
for prof in matching.data_p.index:
    print(f"Профиль {prof}: Студенты, поставившие первым приоритетом: {matching.priorities[prof]}, "
          f"целевиков: {matching.priorities_target[prof]}")

print(f"Размер резерва студентов: {matching.reserve_size}")
print(f"Резерв студентов: {matching.reserve_students}")

# почему-то при вызове matching.reserve_students список пустой, а при вызове matching.fill_reserve() список не пустой. не могу понять, в чём причина :(

Профиль 0: Студенты, поставившие первым приоритетом: 42, целевиков: 0
Профиль 1: Студенты, поставившие первым приоритетом: 48, целевиков: 3
Профиль 2: Студенты, поставившие первым приоритетом: 61, целевиков: 2
Профиль 3: Студенты, поставившие первым приоритетом: 52, целевиков: 2
Профиль 4: Студенты, поставившие первым приоритетом: 50, целевиков: 3
Профиль 5: Студенты, поставившие первым приоритетом: 55, целевиков: 3
Профиль 6: Студенты, поставившие первым приоритетом: 52, целевиков: 4
Профиль 7: Студенты, поставившие первым приоритетом: 48, целевиков: 3
Размер резерва студентов: 18
Резерв студентов: []


In [246]:
for prof, matched_students in matching.profiles_matches.items():
    print(f"Профиль {prof}: Студенты - {matched_students}")
    print(f"Всего на профиле {prof} {len(matched_students)} студентов")

    max_size = matching.data_p.loc[prof, 'max_group_size']
    min_size = matching.data_p.loc[prof, 'min_group_size']
    if len(matched_students) > 0:
        full_groups = len(matched_students) // max_size
        remainder = len(matched_students) % max_size   
        if remainder == 0:
            print(f"Профиль {prof} укомплектован")
        else:
            if remainder >= min_size:
                students_to_remove = remainder - min_size
                print(f"Для минимальной заполненности последней возможной открытой группы профиля {prof} выгнать: {students_to_remove}")
            else:
                print(f"Остаток студентов для профиля {prof} (менее минимального размера группы): {remainder}")
                students_needed_to_min = min_size - remainder
                print(f"Не хватает студентов до минимальной заполненности группы для профиля {prof}: {students_needed_to_min}")         
            students_needed_to_max = max_size - remainder if remainder > 0 else 0
            print(f"Не хватает студентов до максимальной заполненности группы для профиля {prof}: {students_needed_to_max}")
    else:
        print(f"На профиле {prof} нет зачисленных студентов.")

Профиль 0: Студенты - [1, 41, 44, 46, 62, 67, 82, 90, 93, 112, 120, 123, 131, 140, 145, 162, 184, 185, 219, 230, 232, 236, 237, 239, 261, 263, 308, 317, 324, 326, 340, 359, 369, 385, 390, 395, 399, 426, 428, 429, 434, 438]
Всего на профиле 0 42 студентов
Остаток студентов для профиля 0 (менее минимального размера группы): 14
Не хватает студентов до минимальной заполненности группы для профиля 0: 1
Не хватает студентов до максимальной заполненности группы для профиля 0: 14
Профиль 1: Студенты - [4, 5, 12, 13, 14, 47, 49, 50, 52, 66, 77, 80, 91, 92, 101, 122, 135, 137, 149, 159, 170, 171, 174, 177, 181, 182, 194, 202, 213, 214, 216, 220, 233, 242, 245, 265, 267, 293, 304, 311, 323, 325, 350, 351, 391, 398, 400, 404]
Всего на профиле 1 48 студентов
Остаток студентов для профиля 1 (менее минимального размера группы): 18
Не хватает студентов до минимальной заполненности группы для профиля 1: 2
Не хватает студентов до максимальной заполненности группы для профиля 1: 12
Профиль 2: Студенты - 

In [247]:
print("Незачисленные студенты:")
print(matching.unmatched_students)

Незачисленные студенты:
[3, 15, 17, 33, 64, 97, 99, 103, 109, 114, 129, 133, 138, 143, 152, 163, 176, 179, 186, 225, 238, 246, 252, 260, 271, 292, 301, 310, 319, 344, 355, 363, 365, 394, 397, 401, 409, 414, 415, 418, 436, 445]
